In [2]:
%pip install langchain langchain_core langchain_community openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 9.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.9/907.9 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from openai import AzureOpenAI
import pandas as pd
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
import os

/Users/kevin/data-science/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [29]:
### Defining Prompts
# document_prompt = PromptTemplate(
#     input_variables=["page_content"],
#      template="{page_content}"
# )
document_variable_name = "context"
prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)


prompt_refine = PromptTemplate.from_template(
    "Here's your first summary: {prev_response}. "
    "Now add to it based on the following context: {context}"
)


In [36]:
### Generating Dummy Data
dummy = pd.DataFrame({"keys":[" Gather all ingredients. Preheat the oven to 350 degrees F (175 degrees C). Grease and flour a 9-inch square cake pan. "," Cream sugar and butter together in a mixing bowl. Add eggs, one at a time, beating briefly after each addition. Mix in vanilla. ", " Combine flour and baking powder in a separate bowl. Add to the wet ingredients and mix well. Add milk and stir until smooth. "], "values": [1,2,3]})
dummy_docs = DataFrameLoader(dummy,page_content_column="keys").load()

In [38]:
### Loading custom LLM wrapper chain class
import llm
from imp import reload
reload(llm)
summary = llm.chain(initial_prompt=prompt, refine_prompt=prompt_refine, documents=dummy_docs, time_delay_seconds=0)

In [39]:
## summary
print(summary)

The content continues with the baking process, instructing to combine flour and baking powder in a separate bowl. This dry mixture is then added to the previously prepared wet ingredients and mixed thoroughly. The recipe then calls for the addition of milk, which should be stirred into the mixture until it achieves a smooth consistency.


In [4]:
prompt = PromptTemplate.from_template(
            """Given a detailed service event report about an engine or vehicle failure, extract the following information from the text and format the result as a python dictionary with JSON formatting standards. This means surround all properties in double quotes instead of single:

        With the key of "model_identified_complaint", extract the primary customer complaint. The complaint is defined as the main engine or vehicle issue that the customer or technician is claiming is causing the repair, and summarize the complaint in less than 4 words, including the exact part or system affected, using technical terms.
If no such information is found, mention "Not available". in the response, only keep the relevant information. Avoid narriative explainations. Only include engine related terms and complaints. Do not count P codes as a complaint. Prioritize the first mentioned complaint, and if there is something mentioned by the customer prioritize that. If the text contains the phrase "maint" or similar to "maintenance", output the complaint as maintenance. For the outputted complaint, fix any spelling mistakes. If the identified complaint is in another language, translate it to English. Remove any quotations and extra punctuation, just keep the alphanumeric characters. Output the result in lowercase.
With the key of "identified_cause", extract the assignable cause of repair if any are mentioned in the text. For example: Belt noise, no communication, no start, sensor malfunctioning. This cause should identify the component or the cause that was fixed that able to fix the issue during the repair.
only keep the relevant information and avoid narrative explanations.
With the key of "identified_repairs", extract and present the repair actions in a short summary, For example: replaced nox sensor, replaced egr cooler, fixed coolant leak, in the response and only keep the relevant information while avoiding narrative explanations.
With the key of "identified_keywords", extract important service, repair or symptom related keywords in a comma separated list. Focus on any objective nouns that are mentioned. Examples: loose connector, burning smell, P codes or U codes.
in the response, only keep the relevant information and avoid any narrative explanations.
With the key of "identified_codes", given a detailed service event report about an engine or vehicle failure, extract ONLY P codes or U codes that start with a P or U and have 4 digits and truncate values before hyphens if any are present. Codes with hyphens like U0100-00 or P0100-00 should be truncated to U0100, and P0100 respectively. For example text might mention U0100, P2580 etc. If there are no such codes mentioned in Combined Narrative: , return "Not Found"
Do not include the prefix of "P Code and U Code:" in the response, only keep the relevant information and avoid narrative explanations. If none are found, return "Not found".
output the final combined response in the format of a python dictionary, with the keys of the dictionary being the following: "complaint", "cause", "repairs", "keywords", "codes", and the values being the answers to the corresponding information above.

     {context}
     """)


In [6]:
prompt.format(context = "testing")

'Given a detailed service event report about an engine or vehicle failure, extract the following information from the text and format the result as a python dictionary with JSON formatting standards. This means surround all properties in double quotes instead of single:\n\n        With the key of "model_identified_complaint", extract the primary customer complaint. The complaint is defined as the main engine or vehicle issue that the customer or technician is claiming is causing the repair, and summarize the complaint in less than 4 words, including the exact part or system affected, using technical terms.\nIf no such information is found, mention "Not available". in the response, only keep the relevant information. Avoid narriative explainations. Only include engine related terms and complaints. Do not count P codes as a complaint. Prioritize the first mentioned complaint, and if there is something mentioned by the customer prioritize that. If the text contains the phrase "maint" or s